In [1]:
from time import time

import tensorflow as tf
from absl import flags
import scipy.sparse as sp
import numpy as np
from tqdm.autonotebook import tqdm

from PPI.utils import load_edgelist_data, sparse_to_info, process_graph, split_graph_edges
from model.gcn import GCN

/home/veritas/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [2]:
# Set hyper-parameters.
FLAGS = flags.FLAGS
flags.DEFINE_float(name='lr', default=0.001, help='learning rate', lower_bound=0.)
flags.DEFINE_float(name='val_r', default=0.02, help='Validation set ratio', lower_bound=0., upper_bound=1.)
flags.DEFINE_float(name='test_r', default=0.02, help='Test set ratio', lower_bound=0., upper_bound=1.)
flags.DEFINE_integer(name='seed', default=None, help='Seed for data split. Not the seed for model training.')
flags.DEFINE_integer(name='epochs', default=20, help='Number of training epochs.', lower_bound=1)
flags.DEFINE_string(name='logs', default='./logs', 
                    help='root directory for logs, checkpoints, and other records.')
FLAGS.mark_as_parsed()  # This is necessary for using FLAGS in jupyter.

In [3]:
data_path = '../data/yeast.edgelist'
adj = load_edgelist_data(data_path)  # Get adjacency matrix in CSR format.

# The adjacency matrix is symmetric but has non-zero elements on the diagonal.
num_nodes = adj.shape[0]
num_edges = adj.sum()

In [4]:
adj_orig = adj - sp.diags(adj.diagonal())
adj_orig.eliminate_zeros()

In [5]:
adj_train, edges_train, edges_val, edges_val_fake, edges_test, edges_test_fake = \
    split_graph_edges(adj, val_ratio=FLAGS.val_r, test_ratio=FLAGS.test_r, seed=FLAGS.seed)

adj_train_norm = process_graph(adj_train)


Starting splitting!
Starting fake generation for test set! Time: 0s
Starting fake generation for val set! Time: 2s
Finalizing! Time: 3s
(509277, 2) 6525


In [6]:
model = GCN(num_features=num_nodes, num_nonzero_features=num_nodes, h1=num_nodes, h2=num_nodes, dropout=0)
optimizer = tf.keras.optimizers.Adam(learning_rate=FLAGS.lr)
writer = tf.summary.create_file_writer(FLAGS.logs)

In [7]:
adj_labels = adj_train + sp.identity(num_nodes)
one_hot = sparse_to_info(sp.identity(num_nodes))
position_weight = (num_nodes ** 2 - num_edges) / num_edges
norm = num_nodes ** 2 / (2 * (num_nodes ** 2 - num_edges))


In [9]:
with writer.as_default():
    # Training loop
    for epoch in tqdm(range(1, FLAGS.epochs+1)):  # Only 1 training iteration per epoch.
        tic = time()
        inputs = [one_hot, adj_train_norm]
        
        with tf.GradientTape() as tape:  # Begin gradient calculations from here.
            outputs = model(inputs, training=True)        
            loss = norm * tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(
                labels=adj_labels, logits=outputs, pos_weight=position_weight))
        variables = model.trainable_variables
        gradients = tape.gradient(target=loss, sources=variables)
        optimizer.apply_gradients(grads_and_vars=zip(gradients, variables))
        
        toc = int(time() - tic)
        print(f'Epoch {epoch:03d} Time: {toc}s, loss: {float(loss):.3f}')
        tf.summary.scalar(name='Training Loss', data=loss, step=epoch, description='Training loss for each epoch.')
    else:
        writer.flush()




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



RuntimeError: in converted code:
    relative to /home/veritas:

    PycharmProjects/Bioinformatics-Project/model/gcn.py:99 call  *
        outputs = self.hidden1(inputs, training=training)
    anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    PycharmProjects/Bioinformatics-Project/model/gcn.py:50 call  *
        raise RuntimeError('Input tensor does not match sparsity description!')

    RuntimeError: Input tensor does not match sparsity description!
